<a href="https://colab.research.google.com/github/yeshivadataanalytics/predict-alzheimers/blob/zeyu/1_MCI_to_AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re

# 1. Data Processing

In [ ]:
data = pd.read_csv('ADNIMERGE (1).csv')

/Users/zeyu/miniforge3/envs/tensorflow/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (19,20,21,104,105,106) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
data

,RID,COLPROT,ORIGPROT,PTID,SITE,VISCODE,EXAMDATE,DX_bl,AGE,PTGENDER,...,PTAU_bl,FDG_bl,PIB_bl,AV45_bl,FBB_bl,Years_bl,Month_bl,Month,M,update_stamp
0,2,ADNI1,ADNI1,011_S_0002,11,bl,2005-09-08,CN,74.3,Male,...,NaN,1.36665,NaN,NaN,NaN,0.000000,0.00000,0,0,2021-12-08 04:59:14.0
1,3,ADNI1,ADNI1,011_S_0003,11,bl,2005-09-12,AD,81.3,Male,...,22.83,1.08355,NaN,NaN,NaN,0.000000,0.00000,0,0,2021-12-08 04:59:14.0
2,3,ADNI1,ADNI1,011_S_0003,11,m06,2006-03-13,AD,81.3,Male,...,22.83,1.08355,NaN,NaN,NaN,0.498289,5.96721,6,6,2021-12-08 04:59:14.0
3,3,ADNI1,ADNI1,011_S_0003,11,m12,2006-09-12,AD,81.3,Male,...,22.83,1.08355,NaN,NaN,NaN,0.999316,11.96720,12,12,2021-12-08 04:59:14.0
4,3,ADNI1,ADNI1,011_S_0003,11,m24,2007-09-12,AD,81.3,Male,...,22.83,1.08355,NaN,NaN,NaN,1.998630,23.93440,24,24,2021-12-08 04:59:14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15775,4149,ADNI3,ADNI2,031_S_4149,31,m114,2020-12-07,EMCI,73.3,Male,...,18.54,1.31762,NaN,1.157,NaN,9.303220,111.41000,114,114,2022-02-10 05:01:06.0
15776,4149,ADNI3,ADNI2,031_S_4149,31,m126,2021-11-20,EMCI,73.3,Male,...,18.54,1.31762,NaN,1.157,NaN,10.256000,122.82000,120,126,2022-02-10 05:01:06.0
15777,679,ADNI3,ADNI1,041_S_0679,41,m186,2022-02-08,LMCI,62.9,Male,...,NaN,1.42776,NaN,NaN,NaN,15.515400,185.80300,186,186,2022-02-12 05:00:41.0
15778,6147,ADNI3,ADNI3,127_S_6147,127,m48,2022-02-09,SMC,69.6,Male,...,NaN,NaN,NaN,NaN,1.1242,3.969880,47.54100,48,48,2022-02-12 05:00:58.0


In [ ]:
data.EXAMDATE = pd.to_datetime(data.EXAMDATE)
data

,RID,COLPROT,ORIGPROT,PTID,SITE,VISCODE,EXAMDATE,DX_bl,AGE,PTGENDER,...,PTAU_bl,FDG_bl,PIB_bl,AV45_bl,FBB_bl,Years_bl,Month_bl,Month,M,update_stamp
0,2,ADNI1,ADNI1,011_S_0002,11,bl,2005-09-08,CN,74.3,Male,...,NaN,1.36665,NaN,NaN,NaN,0.000000,0.00000,0,0,2021-12-08 04:59:14.0
1,3,ADNI1,ADNI1,011_S_0003,11,bl,2005-09-12,AD,81.3,Male,...,22.83,1.08355,NaN,NaN,NaN,0.000000,0.00000,0,0,2021-12-08 04:59:14.0
2,3,ADNI1,ADNI1,011_S_0003,11,m06,2006-03-13,AD,81.3,Male,...,22.83,1.08355,NaN,NaN,NaN,0.498289,5.96721,6,6,2021-12-08 04:59:14.0
3,3,ADNI1,ADNI1,011_S_0003,11,m12,2006-09-12,AD,81.3,Male,...,22.83,1.08355,NaN,NaN,NaN,0.999316,11.96720,12,12,2021-12-08 04:59:14.0
4,3,ADNI1,ADNI1,011_S_0003,11,m24,2007-09-12,AD,81.3,Male,...,22.83,1.08355,NaN,NaN,NaN,1.998630,23.93440,24,24,2021-12-08 04:59:14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15775,4149,ADNI3,ADNI2,031_S_4149,31,m114,2020-12-07,EMCI,73.3,Male,...,18.54,1.31762,NaN,1.157,NaN,9.303220,111.41000,114,114,2022-02-10 05:01:06.0
15776,4149,ADNI3,ADNI2,031_S_4149,31,m126,2021-11-20,EMCI,73.3,Male,...,18.54,1.31762,NaN,1.157,NaN,10.256000,122.82000,120,126,2022-02-10 05:01:06.0
15777,679,ADNI3,ADNI1,041_S_0679,41,m186,2022-02-08,LMCI,62.9,Male,...,NaN,1.42776,NaN,NaN,NaN,15.515400,185.80300,186,186,2022-02-12 05:00:41.0
15778,6147,ADNI3,ADNI3,127_S_6147,127,m48,2022-02-09,SMC,69.6,Male,...,NaN,NaN,NaN,NaN,1.1242,3.969880,47.54100,48,48,2022-02-12 05:00:58.0


In [ ]:
data.DX.unique()

array(['CN', 'Dementia', 'MCI', nan], dtype=object)

In [ ]:
data1 = data[['RID', 'PTID', 'EXAMDATE', 'DX', 'AGE']]

In [ ]:
data1

,RID,PTID,EXAMDATE,DX,AGE
0,2,011_S_0002,2005-09-08,CN,74.3
1,3,011_S_0003,2005-09-12,Dementia,81.3
2,3,011_S_0003,2006-03-13,Dementia,81.3
3,3,011_S_0003,2006-09-12,Dementia,81.3
4,3,011_S_0003,2007-09-12,Dementia,81.3
...,...,...,...,...,...
15775,4149,031_S_4149,2020-12-07,NaN,73.3
15776,4149,031_S_4149,2021-11-20,NaN,73.3
15777,679,041_S_0679,2022-02-08,NaN,62.9
15778,6147,127_S_6147,2022-02-09,NaN,69.6


## 1.1 Selected data which patient convert from MCI to ad

In [ ]:
data2 = data1.pivot_table(index='PTID', columns='DX', values='EXAMDATE')
data2

DX,CN,Dementia,MCI
PTID,,,
002_S_0295,2009-01-30 06:00:00.000000000,NaT,NaT
002_S_0413,2011-10-06 07:23:04.615384576,NaT,NaT
002_S_0559,2007-10-23 19:12:00.000000000,NaT,NaT
002_S_0619,NaT,2007-05-16 12:00:00,NaT
002_S_0685,2009-04-02 12:00:00.000000000,NaT,NaT
...,...,...,...
941_S_6581,2019-09-14 00:00:00.000000000,NaT,NaT
941_S_6803,NaT,NaT,2019-09-17
941_S_6854,NaT,2020-09-08 12:00:00,NaT


In [ ]:
data2 = data2[data2.CN.isna()]

In [ ]:
data2

DX,CN,Dementia,MCI
PTID,,,
002_S_0619,NaT,2007-05-16 12:00:00,NaT
002_S_0729,NaT,2010-04-23 00:00:00,2006-11-12 00:00:00
002_S_0782,NaT,NaT,2008-02-02 08:00:00
002_S_0816,NaT,2007-05-03 00:00:00,NaT
002_S_0938,NaT,2007-09-03 06:00:00,NaT
...,...,...,...
941_S_6068,NaT,2021-08-18 00:00:00,2018-08-21 00:00:00
941_S_6345,NaT,2020-01-07 00:00:00,2018-05-14 00:00:00
941_S_6803,NaT,NaT,2019-09-17 00:00:00


In [ ]:
data3 = data2[(~data2.Dementia.isna()) & (~data2.MCI.isna())]
data3

DX,CN,Dementia,MCI
PTID,,,
002_S_0729,NaT,2010-04-23 00:00:00,2006-11-12 00:00:00
002_S_0954,NaT,2008-05-16 00:00:00,2007-02-08 00:00:00
002_S_1070,NaT,2009-03-02 00:00:00,2007-06-08 00:00:00
002_S_1268,NaT,2013-03-14 00:00:00,2009-05-09 03:00:00
002_S_4171,NaT,2013-10-29 00:00:00,2012-02-24 00:00:00
...,...,...,...
941_S_1295,NaT,2008-09-04 00:00:00,2007-09-03 16:00:00
941_S_1311,NaT,2008-10-06 08:00:00,2007-06-18 00:00:00
941_S_1363,NaT,2007-10-10 00:00:00,2007-03-26 00:00:00


In [ ]:
data3['days'] = data3.Dementia-data3.MCI
data3

/var/folders/0f/brs6kj9d6bbftjh0sk1763yh0000gn/T/ipykernel_1087/3770156558.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['days'] = data3.Dementia-data3.MCI


DX,CN,Dementia,MCI,days
PTID,,,,
002_S_0729,NaT,2010-04-23 00:00:00,2006-11-12 00:00:00,1258 days 00:00:00
002_S_0954,NaT,2008-05-16 00:00:00,2007-02-08 00:00:00,463 days 00:00:00
002_S_1070,NaT,2009-03-02 00:00:00,2007-06-08 00:00:00,633 days 00:00:00
002_S_1268,NaT,2013-03-14 00:00:00,2009-05-09 03:00:00,1404 days 21:00:00
002_S_4171,NaT,2013-10-29 00:00:00,2012-02-24 00:00:00,613 days 00:00:00
...,...,...,...,...
941_S_1295,NaT,2008-09-04 00:00:00,2007-09-03 16:00:00,366 days 08:00:00
941_S_1311,NaT,2008-10-06 08:00:00,2007-06-18 00:00:00,476 days 08:00:00
941_S_1363,NaT,2007-10-10 00:00:00,2007-03-26 00:00:00,198 days 00:00:00


In [ ]:
data3.days = data3.days.apply(lambda x: int(str(x).split(' ')[0]))
data3

/Users/zeyu/miniforge3/envs/tensorflow/lib/python3.9/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


DX,CN,Dementia,MCI,days
PTID,,,,
002_S_0729,NaT,2010-04-23 00:00:00,2006-11-12 00:00:00,1258
002_S_0954,NaT,2008-05-16 00:00:00,2007-02-08 00:00:00,463
002_S_1070,NaT,2009-03-02 00:00:00,2007-06-08 00:00:00,633
002_S_1268,NaT,2013-03-14 00:00:00,2009-05-09 03:00:00,1404
002_S_4171,NaT,2013-10-29 00:00:00,2012-02-24 00:00:00,613
...,...,...,...,...
941_S_1295,NaT,2008-09-04 00:00:00,2007-09-03 16:00:00,366
941_S_1311,NaT,2008-10-06 08:00:00,2007-06-18 00:00:00,476
941_S_1363,NaT,2007-10-10 00:00:00,2007-03-26 00:00:00,198


In [ ]:
data_MCI_AD = data3[data3.days>365]
data_MCI_AD

DX,CN,Dementia,MCI,days
PTID,,,,
002_S_0729,NaT,2010-04-23 00:00:00,2006-11-12 00:00:00,1258
002_S_0954,NaT,2008-05-16 00:00:00,2007-02-08 00:00:00,463
002_S_1070,NaT,2009-03-02 00:00:00,2007-06-08 00:00:00,633
002_S_1268,NaT,2013-03-14 00:00:00,2009-05-09 03:00:00,1404
002_S_4171,NaT,2013-10-29 00:00:00,2012-02-24 00:00:00,613
...,...,...,...,...
168_S_6634,NaT,2020-08-22 00:00:00,2019-02-21 00:00:00,548
941_S_1295,NaT,2008-09-04 00:00:00,2007-09-03 16:00:00,366
941_S_1311,NaT,2008-10-06 08:00:00,2007-06-18 00:00:00,476


- People who were detected converted from MCI to Dementia have 125 after 3 years.
- People who were detected converted from MCI to Dementia have 221 after 2 years.
- People who were detected converted from MCI to Dementia have 331 after 1 years.

## 1.2 Selected data which patient convert from MCI to MCI

In [ ]:
data1

,RID,PTID,EXAMDATE,DX,AGE
0,2,011_S_0002,2005-09-08,CN,74.3
1,3,011_S_0003,2005-09-12,Dementia,81.3
2,3,011_S_0003,2006-03-13,Dementia,81.3
3,3,011_S_0003,2006-09-12,Dementia,81.3
4,3,011_S_0003,2007-09-12,Dementia,81.3
...,...,...,...,...,...
15775,4149,031_S_4149,2020-12-07,NaN,73.3
15776,4149,031_S_4149,2021-11-20,NaN,73.3
15777,679,041_S_0679,2022-02-08,NaN,62.9
15778,6147,127_S_6147,2022-02-09,NaN,69.6


In [ ]:
data4 = data1[data1.DX=='MCI']
data4 = data4.reset_index(drop=True)
data4

,RID,PTID,EXAMDATE,DX,AGE
0,4,022_S_0004,2005-11-08,MCI,67.5
1,4,022_S_0004,2006-05-02,MCI,67.5
2,4,022_S_0004,2006-11-14,MCI,67.5
3,4,022_S_0004,2007-05-14,MCI,67.5
4,4,022_S_0004,2008-11-18,MCI,67.5
...,...,...,...,...,...
4810,4272,007_S_4272,2022-01-24,MCI,70.9
4811,2245,027_S_2245,2022-01-27,MCI,57.0
4812,4919,027_S_4919,2022-01-31,MCI,74.8
4813,6635,019_S_6635,2022-01-28,MCI,56.5


In [ ]:
data4['diff'] = data4.groupby('PTID')['EXAMDATE'].diff()
data4

,RID,PTID,EXAMDATE,DX,AGE,diff
0,4,022_S_0004,2005-11-08,MCI,67.5,NaT
1,4,022_S_0004,2006-05-02,MCI,67.5,175 days
2,4,022_S_0004,2006-11-14,MCI,67.5,196 days
3,4,022_S_0004,2007-05-14,MCI,67.5,181 days
4,4,022_S_0004,2008-11-18,MCI,67.5,554 days
...,...,...,...,...,...,...
4810,4272,007_S_4272,2022-01-24,MCI,70.9,1112 days
4811,2245,027_S_2245,2022-01-27,MCI,57.0,1485 days
4812,4919,027_S_4919,2022-01-31,MCI,74.8,1509 days
4813,6635,019_S_6635,2022-01-28,MCI,56.5,374 days


#### Keep the first one and the last one according to PTID

In [ ]:
index = pd.concat([data4.PTID.drop_duplicates(keep='first'), data4.PTID.drop_duplicates(keep='last')]).index

In [ ]:
data5 = data4.loc[index]
data5 = data5.sort_values(by='RID')
data5 = data5.reset_index(drop=True)
data5

,RID,PTID,EXAMDATE,DX,AGE
0,2,011_S_0002,2017-10-18,MCI,74.3
1,2,011_S_0002,2012-09-26,MCI,74.3
2,4,022_S_0004,2005-11-08,MCI,67.5
3,4,022_S_0004,2008-11-18,MCI,67.5
4,6,100_S_0006,2007-11-08,MCI,80.4
...,...,...,...,...,...
2409,7020,067_S_7020,2021-11-22,MCI,67.1
2410,7023,036_S_7023,2021-11-16,MCI,76.7
2411,7023,036_S_7023,2021-11-16,MCI,76.7
2412,7025,024_S_7025,2021-11-18,MCI,68.3


#### delete data which appear only once

In [ ]:
data5.groupby('PTID').filter(lambda x: len(x)>1)

,RID,PTID,EXAMDATE,DX,AGE
0,2,011_S_0002,2017-10-18,MCI,74.3
1,2,011_S_0002,2012-09-26,MCI,74.3
2,4,022_S_0004,2005-11-08,MCI,67.5
3,4,022_S_0004,2008-11-18,MCI,67.5
4,6,100_S_0006,2007-11-08,MCI,80.4
...,...,...,...,...,...
2409,7020,067_S_7020,2021-11-22,MCI,67.1
2410,7023,036_S_7023,2021-11-16,MCI,76.7
2411,7023,036_S_7023,2021-11-16,MCI,76.7
2412,7025,024_S_7025,2021-11-18,MCI,68.3


In [ ]:
data5['diff'] = data5.groupby('PTID')['EXAMDATE'].diff()
data5

,RID,PTID,EXAMDATE,DX,AGE,diff
0,2,011_S_0002,2017-10-18,MCI,74.3,NaT
1,2,011_S_0002,2012-09-26,MCI,74.3,-1848 days
2,4,022_S_0004,2005-11-08,MCI,67.5,NaT
3,4,022_S_0004,2008-11-18,MCI,67.5,1106 days
4,6,100_S_0006,2007-11-08,MCI,80.4,NaT
...,...,...,...,...,...,...
2409,7020,067_S_7020,2021-11-22,MCI,67.1,0 days
2410,7023,036_S_7023,2021-11-16,MCI,76.7,NaT
2411,7023,036_S_7023,2021-11-16,MCI,76.7,0 days
2412,7025,024_S_7025,2021-11-18,MCI,68.3,NaT


In [ ]:
data5 = data5[pd.notnull(data5['diff'])]
data5

,RID,PTID,EXAMDATE,DX,AGE,diff
1,2,011_S_0002,2012-09-26,MCI,74.3,-1848 days
3,4,022_S_0004,2008-11-18,MCI,67.5,1106 days
5,6,100_S_0006,2005-11-29,MCI,80.4,-709 days
7,8,011_S_0008,2015-10-29,MCI,84.5,0 days
9,15,100_S_0015,2011-11-03,MCI,80.8,0 days
...,...,...,...,...,...,...
2405,7015,126_S_7015,2021-11-30,MCI,77.1,0 days
2407,7016,019_S_7016,2021-10-25,MCI,66.1,0 days
2409,7020,067_S_7020,2021-11-22,MCI,67.1,0 days
2411,7023,036_S_7023,2021-11-16,MCI,76.7,0 days


In [ ]:
def dataClean(x):
    pattern = re.compile(r'[0-9]+')
    data_number = re.findall(pattern, x)
    return int(data_number[0])

In [ ]:
data5['diff'] = data5['diff'].apply(lambda x: dataClean(str(x)))
data5

/var/folders/0f/brs6kj9d6bbftjh0sk1763yh0000gn/T/ipykernel_1087/3830338665.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data5['diff'] = data5['diff'].apply(lambda x: dataClean(str(x)))


,RID,PTID,EXAMDATE,DX,AGE,diff
1,2,011_S_0002,2012-09-26,MCI,74.3,1848
3,4,022_S_0004,2008-11-18,MCI,67.5,1106
5,6,100_S_0006,2005-11-29,MCI,80.4,709
7,8,011_S_0008,2015-10-29,MCI,84.5,0
9,15,100_S_0015,2011-11-03,MCI,80.8,0
...,...,...,...,...,...,...
2405,7015,126_S_7015,2021-11-30,MCI,77.1,0
2407,7016,019_S_7016,2021-10-25,MCI,66.1,0
2409,7020,067_S_7020,2021-11-22,MCI,67.1,0
2411,7023,036_S_7023,2021-11-16,MCI,76.7,0


In [ ]:
data5 = data5[data5['diff']>365]
data5 = data5.reset_index(drop=True)
data5

,RID,PTID,EXAMDATE,DX,AGE,diff
0,2,011_S_0002,2012-09-26,MCI,74.3,1848
1,4,022_S_0004,2008-11-18,MCI,67.5,1106
2,6,100_S_0006,2005-11-29,MCI,80.4,709
3,51,099_S_0051,2005-12-08,MCI,66.5,1099
4,55,018_S_0055,2012-05-01,MCI,75.5,537
...,...,...,...,...,...,...
711,6885,036_S_6885,2021-10-05,MCI,71.7,378
712,6887,036_S_6887,2021-10-19,MCI,72.9,379
713,6888,123_S_6888,2021-10-18,MCI,63.7,376
714,6889,033_S_6889,2020-10-23,MCI,67.8,367


In [ ]:
data_MCI_MCI = data5
data_MCI_MCI

,RID,PTID,EXAMDATE,DX,AGE,diff
0,2,011_S_0002,2012-09-26,MCI,74.3,1848
1,4,022_S_0004,2008-11-18,MCI,67.5,1106
2,6,100_S_0006,2005-11-29,MCI,80.4,709
3,51,099_S_0051,2005-12-08,MCI,66.5,1099
4,55,018_S_0055,2012-05-01,MCI,75.5,537
...,...,...,...,...,...,...
711,6885,036_S_6885,2021-10-05,MCI,71.7,378
712,6887,036_S_6887,2021-10-19,MCI,72.9,379
713,6888,123_S_6888,2021-10-18,MCI,63.7,376
714,6889,033_S_6889,2020-10-23,MCI,67.8,367


Now we have two dataset. 
- data_MCI_AD stands for patient converted from MCI to AD
- data_MCI_MCI stands for patient converted from MCI to MCI